<a href="https://colab.research.google.com/github/MrShiroLu/AnkaraSuNumunesiRotaOptimizasyonu/blob/main/AnkaraGolleriNumuneToplamaEnKisaRota.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Karınca Kolonisi Algoritması ile Ankara Gölet Rota Optimizasyonu

## Proje Açıklaması
Çevre Bakanlığı birimlerinin Ankara'daki 10 farklı göletten su numunesi toplaması için **en kısa rotanın** hesaplanması.

### Kullanılan Teknolojiler:
- **Google Maps API**: Gerçek koordinatlar ve mesafeler
- **Ant Colony Optimization (ACO)**: Karınca kolonisi algoritması
- **Python**: NumPy, Pandas, Matplotlib, Folium

### Çözüm Yaklaşımı:
1. Google Maps API ile göletlerin gerçek koordinatlarını al
2. Distance Matrix API ile tüm noktalar arası mesafeleri hesapla
3. ACO algoritması ile optimal rotayı bul
4. Sonuçları görselleştir ve raporla


## 1. Kütüphaneleri Yükle ve Ayarla


In [ ]:
# Gerekli kütüphaneleri yükle
import googlemaps
import folium
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from math import radians, sin, cos, sqrt, atan2
import warnings
warnings.filterwarnings('ignore')

# Görselleştirme ayarları
plt.style.use('default')
sns.set_palette("husl")

print(" Kütüphaneler yüklendi!")
print(f" Tarih: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


## 2. Google Maps API Bağlantısı


In [ ]:
# Google Maps API Anahtarı
# API anahtarı almak için: https://console.cloud.google.com/apis/credentials
# Gerekli API'ler: Geocoding API, Distance Matrix API

API_KEY = "API_ANAHTARINIZI_BURAYA_YAPISTIRIN"

# Google Maps istemcisi
gmaps = googlemaps.Client(key=API_KEY)

print(" Google Maps API bağlantısı kuruldu!")


## 3. Göletlerin Tanımlanması


In [ ]:
# Ankara'daki göletler ve barajlar
goletler = [
    "Eymir Gölü, Ankara",
    "Mogan Gölü, Gölbaşı, Ankara",
    "Çubuk Barajı, Çubuk, Ankara",
    "Kurtboğazı Barajı, Kızılcahamam, Ankara",
    "Bayındır Barajı, Ankara",
    "Kesikköprü Barajı, Ankara",
    "İmrahor Vadisi Göleti, Ankara",
    "Altınpark Gölü, Altındağ, Ankara",
    "Harikalar Diyarı Göleti, Keçiören, Ankara",
    "Göksu Parkı Göleti, Eryaman, Ankara"
]

# Başlangıç noktası (Çevre Bakanlığı)
baslangic = "Çevre, Şehircilik ve İklim Değişikliği Bakanlığı, Ankara"

print(f" Toplam Gölet Sayısı: {len(goletler)}")
print(f" Başlangıç Noktası: {baslangic}\n")
print(" GÖLET LİSTESİ:")
print("="*70)
for i, golet in enumerate(goletler, 1):
    print(f"{i:2d}. {golet}")
print("="*70)


## 4. Koordinatların Alınması (Google Maps API)


In [ ]:
def get_coordinates(address):
    """Google Maps Geocoding API ile koordinat al"""
    try:
        result = gmaps.geocode(address)
        if result:
            location = result[0]['geometry']['location']
            return {
                'lat': location['lat'],
                'lng': location['lng'],
                'address': result[0]['formatted_address']
            }
    except Exception as e:
        print(f" Hata: {e}")
    return None

# Başlangıç koordinatı
print(" Koordinatlar alınıyor...\n")
baslangic_koord = get_coordinates(baslangic)
print(f" {baslangic}")
print(f"    {baslangic_koord['lat']:.6f}, {baslangic_koord['lng']:.6f}\n")

# Gölet koordinatları
golet_data = []
for i, golet in enumerate(goletler, 1):
    koord = get_coordinates(golet)
    if koord:
        golet_data.append({
            'id': i,
            'name': golet,
            'lat': koord['lat'],
            'lng': koord['lng'],
            'address': koord['address']
        })
        print(f"{i:2d}. {golet}")
        print(f"     {koord['lat']:.6f}, {koord['lng']:.6f}")

df_goletler = pd.DataFrame(golet_data)
print(f"\n {len(df_goletler)} göletin koordinatları alındı!")


## 5. Mesafe Matrisinin Oluşturulması


In [ ]:
def haversine_distance(coord1, coord2):
    """Kuş uçuşu mesafe (km)"""
    R = 6371
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

def create_distance_matrix(origins, destinations):
    """Google Maps Distance Matrix API ile mesafe matrisi"""
    n = len(origins)
    distance_matrix = np.zeros((n, n))
    time_matrix = np.zeros((n, n))

    print(" Mesafe matrisi hesaplanıyor...\n")

    for i in range(n):
        print(f" {i+1}/{n}", end=" ")
        try:
            result = gmaps.distance_matrix(
                origins=[origins[i]],
                destinations=destinations,
                mode="driving",
                language="tr",
                units="metric"
            )

            if result['status'] == 'OK':
                for j, element in enumerate(result['rows'][0]['elements']):
                    if element['status'] == 'OK':
                        distance_matrix[i][j] = element['distance']['value'] / 1000  # km
                        time_matrix[i][j] = element['duration']['value'] / 60  # dk
                    else:
                        distance_matrix[i][j] = haversine_distance(origins[i], destinations[j])
                        time_matrix[i][j] = distance_matrix[i][j] * 1.5
                print("")
            else:
                print("")
                for j in range(n):
                    distance_matrix[i][j] = haversine_distance(origins[i], destinations[j])
                    time_matrix[i][j] = distance_matrix[i][j] * 1.5
        except:
            print("")
            for j in range(n):
                distance_matrix[i][j] = haversine_distance(origins[i], destinations[j])
                time_matrix[i][j] = distance_matrix[i][j] * 1.5

    return distance_matrix, time_matrix

# Tüm koordinatlar
all_coords = [(baslangic_koord['lat'], baslangic_koord['lng'])]
for _, row in df_goletler.iterrows():
    all_coords.append((row['lat'], row['lng']))

# Mesafe matrisi oluştur
distance_matrix, time_matrix = create_distance_matrix(all_coords, all_coords)

print(f"\n Mesafe matrisi oluşturuldu!")
print(f" Boyut: {distance_matrix.shape}")
print(f" Min mesafe: {distance_matrix[distance_matrix > 0].min():.2f} km")
print(f" Max mesafe: {distance_matrix.max():.2f} km")


## 6. Karınca Kolonisi Algoritması (ACO) Sınıfı


In [ ]:
class AntColonyOptimizer:
    """Karınca Kolonisi Optimizasyonu (ACO) ile TSP Çözümü"""

    def __init__(self, distance_matrix, n_ants=30, n_iterations=100,
                 alpha=1.0, beta=3.0, evaporation_rate=0.3, Q=100):
        self.distance_matrix = distance_matrix
        self.n_cities = len(distance_matrix)
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.alpha = alpha  # Feromon önemi
        self.beta = beta    # Mesafe önemi
        self.evaporation_rate = evaporation_rate
        self.Q = Q

        # Feromon matrisi (başlangıçta eşit)
        self.pheromone = np.ones((self.n_cities, self.n_cities)) / self.n_cities

        # En iyi çözüm
        self.best_route = None
        self.best_distance = float('inf')
        self.best_distance_history = []

    def calculate_route_distance(self, route):
        """Rota mesafesini hesapla"""
        distance = 0
        for i in range(len(route) - 1):
            distance += self.distance_matrix[route[i]][route[i+1]]
        distance += self.distance_matrix[route[-1]][route[0]]  # Başa dön
        return distance

    def construct_solution(self, start=0):
        """Bir karınca için rota oluştur (olasılıksal)"""
        route = [start]
        unvisited = set(range(self.n_cities)) - {start}

        while unvisited:
            current = route[-1]

            # Her şehir için olasılık hesapla
            probabilities = []
            for city in unvisited:
                pheromone = self.pheromone[current][city] ** self.alpha
                heuristic = (1.0 / self.distance_matrix[current][city]) ** self.beta
                probabilities.append(pheromone * heuristic)

            # Normalize et
            probabilities = np.array(probabilities)
            probabilities = probabilities / probabilities.sum()

            # Olasılıklara göre seç
            next_city = np.random.choice(list(unvisited), p=probabilities)

            route.append(next_city)
            unvisited.remove(next_city)

        return route

    def update_pheromones(self, all_routes, all_distances):
        """Feromonları güncelle (buharlaşma + yeni feromon)"""
        # Buharlaşma
        self.pheromone *= (1 - self.evaporation_rate)

        # Yeni feromon ekle
        for route, distance in zip(all_routes, all_distances):
            pheromone_deposit = self.Q / distance

            for i in range(len(route) - 1):
                self.pheromone[route[i]][route[i+1]] += pheromone_deposit
                self.pheromone[route[i+1]][route[i]] += pheromone_deposit

            # Başa dönüş
            self.pheromone[route[-1]][route[0]] += pheromone_deposit
            self.pheromone[route[0]][route[-1]] += pheromone_deposit

    def optimize(self, start=0):
        """Optimizasyonu çalıştır"""
        print("\n KARINCA KOLONİSİ ALGORİTMASI")
        print("="*70)
        print(f"Karınca Sayısı: {self.n_ants}")
        print(f"İterasyon: {self.n_iterations}")
        print(f"Alpha (α): {self.alpha}, Beta (β): {self.beta}")
        print(f"Buharlaşma (ρ): {self.evaporation_rate}, Q: {self.Q}")
        print("="*70 + "\n")

        for iteration in range(self.n_iterations):
            all_routes = []
            all_distances = []

            # Her karınca bir rota oluşturur
            for ant in range(self.n_ants):
                route = self.construct_solution(start)
                distance = self.calculate_route_distance(route)

                all_routes.append(route)
                all_distances.append(distance)

                # En iyi rotayı güncelle
                if distance < self.best_distance:
                    self.best_distance = distance
                    self.best_route = route[:]

            # Feromonları güncelle
            self.update_pheromones(all_routes, all_distances)

            # Geçmişi kaydet
            self.best_distance_history.append(self.best_distance)

            # Her 10 iterasyonda göster
            if (iteration + 1) % 10 == 0:
                avg_dist = np.mean(all_distances)
                print(f"İter {iteration+1:3d}: En İyi={self.best_distance:6.2f} km, Ort={avg_dist:6.2f} km")

        print(f"\n Optimizasyon tamamlandı!")
        print(f" En İyi Mesafe: {self.best_distance:.2f} km\n")

        # Başa dönüş ekle
        final_route = self.best_route + [start]
        return final_route, self.best_distance

print(" ACO sınıfı tanımlandı!")


In [ ]:
# ACO oluştur ve çalıştır
aco = AntColonyOptimizer(
    distance_matrix=distance_matrix,
    n_ants=30,
    n_iterations=100,
    alpha=1.0,
    beta=3.0,
    evaporation_rate=0.3,
    Q=100
)

# Optimizasyonu başlat
optimal_route, total_distance = aco.optimize(start=0)


## 8. Sonuçların Analizi


In [ ]:
# Rota detayları
print("="*80)
print("  OPTIMAL ROTA")
print("="*80 + "\n")

total_time = 0

for i, idx in enumerate(optimal_route):
    if idx == 0:
        if i == 0:
            print(f"{i+1}.  BAŞLANGIÇ: {baslangic}")
            print(f"    {baslangic_koord['lat']:.6f}, {baslangic_koord['lng']:.6f}")
        else:
            print(f"\n{i+1}.  BİTİŞ: {baslangic}")
    else:
        golet = df_goletler.iloc[idx - 1]
        print(f"\n{i+1}.   {golet['name']}")
        print(f"    {golet['lat']:.6f}, {golet['lng']:.6f}")

    if i < len(optimal_route) - 1:
        next_idx = optimal_route[i + 1]
        dist = distance_matrix[idx][next_idx]
        time = time_matrix[idx][next_idx]
        total_time += time
        print(f"   -> {dist:.2f} km ({time:.0f} dk)")

print("\n" + "="*80)
print(f" TOPLAM MESAFE: {total_distance:.2f} km")
print(f"  TOPLAM SÜRE: {total_time:.0f} dakika ({total_time/60:.1f} saat)")
print(f"  GÖLET SAYISI: {len(goletler)}")
print("="*80)


## 9. Yakınsama Grafiği


In [ ]:
# Yakınsama grafiği
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Sol: Mesafe değişimi
ax1.plot(aco.best_distance_history, linewidth=2, color='#2E86AB')
ax1.set_xlabel('İterasyon', fontsize=12, fontweight='bold')
ax1.set_ylabel('En İyi Mesafe (km)', fontsize=12, fontweight='bold')
ax1.set_title(' ACO Yakınsama Grafiği', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.axhline(y=total_distance, color='red', linestyle='--',
            label=f'Optimal: {total_distance:.2f} km')
ax1.legend()

# Sağ: İyileşme %
improvement = [(aco.best_distance_history[0] - d) / aco.best_distance_history[0] * 100
               for d in aco.best_distance_history]
ax2.plot(improvement, linewidth=2, color='#A23B72')
ax2.set_xlabel('İterasyon', fontsize=12, fontweight='bold')
ax2.set_ylabel('İyileşme (%)', fontsize=12, fontweight='bold')
ax2.set_title(' İyileşme Oranı', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.axhline(y=improvement[-1], color='green', linestyle='--',
            label=f'Toplam: {improvement[-1]:.1f}%')
ax2.legend()

plt.tight_layout()
plt.savefig('aco_yakinasma.png', dpi=300, bbox_inches='tight')
plt.show()

print(f" İlk: {aco.best_distance_history[0]:.2f} km")
print(f" Son: {aco.best_distance_history[-1]:.2f} km")
print(f" İyileşme: {improvement[-1]:.1f}%")


## 10. İnteraktif Harita


In [ ]:
# İnteraktif harita oluştur
m = folium.Map(location=[39.9334, 32.8597], zoom_start=10, tiles='OpenStreetMap')

route_coords = []

for i, idx in enumerate(optimal_route):
    coord = all_coords[idx]
    route_coords.append([coord[0], coord[1]])

    if idx == 0:
        # Başlangıç/Bitiş
        if i == 0:
            color, icon, label = 'green', 'play', 'BAŞLANGIÇ'
        else:
            color, icon, label = 'red', 'stop', 'BİTİŞ'

        folium.Marker(
            location=[coord[0], coord[1]],
            popup=f"<b>{label}</b><br>{baslangic}",
            tooltip=label,
            icon=folium.Icon(color=color, icon=icon, prefix='fa')
        ).add_to(m)
    else:
        # Gölet
        golet = df_goletler.iloc[idx - 1]

        if i > 0:
            prev_idx = optimal_route[i-1]
            dist = distance_matrix[prev_idx][idx]
            time = time_matrix[prev_idx][idx]
            info = f"<br><b>Mesafe:</b> {dist:.2f} km<br><b>Süre:</b> {time:.0f} dk"
        else:
            info = ""

        popup_text = f"<b> Durak {i}</b><br>{golet['name']}{info}"

        folium.Marker(
            location=[coord[0], coord[1]],
            popup=folium.Popup(popup_text, max_width=300),
            tooltip=f"Durak {i}: {golet['name']}",
            icon=folium.Icon(color='blue', icon='tint', prefix='fa')
        ).add_to(m)

# Rota çizgisi
folium.PolyLine(
    route_coords,
    color='#E63946',
    weight=4,
    opacity=0.8,
    popup=f'Toplam: {total_distance:.2f} km'
).add_to(m)

# Kaydet
m.save('ankara_golet_rota_haritasi.html')
print(" Harita kaydedildi: ankara_golet_rota_haritasi.html")

m


## 11. Sonuç ve Öneriler


In [ ]:
print("\n" + ""*40)
print("\n" + " "*10 + " PROJE SONUÇLARI ")
print("\n" + ""*40 + "\n")

print("="*80)
print(" PERFORMANS METRİKLERİ")
print("="*80)
print(f" Toplam Mesafe: {total_distance:.2f} km")
print(f"  Seyahat Süresi: {total_time:.0f} dakika ({total_time/60:.1f} saat)")
print(f"  Gölet Sayısı: {len(goletler)}")
print(f" Ortalama Mesafe: {total_distance/len(goletler):.2f} km/gölet")

# Numune toplama süresi ekle
numune_suresi = 15 * len(goletler)  # 15 dk/gölet
toplam_operasyon = total_time + numune_suresi

print(f"\n OPERASYON ÖNERİLERİ:")
print(f"   • Numune Toplama Süresi: {numune_suresi} dakika ({numune_suresi/60:.1f} saat)")
print(f"   • TOPLAM OPERASYON: {toplam_operasyon:.0f} dakika ({toplam_operasyon/60:.1f} saat)")
print(f"   • Sabah 07:00'de başlanması önerilir")
print(f"   • En az 2 kişilik ekip gereklidir")
print(f"   • Soğutmalı numune taşıma ekipmanı (4°C)")

print("="*80)